In [6]:
# importar paquetes a ocupar
import pandas as pd
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from keras import models
import itertools

In [4]:
model = models.load_model('models/cnn_model_3.h5')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 318)               101442    
                                                                 
 dense_25 (Dense)            (None, 318)               101442    
                                                                 
 dense_26 (Dense)            (None, 159)               50721     
                                                                 
 dense_27 (Dense)            (None, 1)                 160       
                                                                 
Total params: 253,765
Trainable params: 253,765
Non-trainable params: 0
_________________________________________________________________


In [5]:
battlelog = ks.read_parquet('datasets/teams/battlelog_train.parquet')
battlelog.head()

,winner,barriers,barriers_center,bushes,bushes_center,waterProp,event_mode,event_map,battle_player1_brawler_name,battle_player2_brawler_name,battle_player3_brawler_name,battle_player1_brawler_Class,battle_player2_brawler_Class,battle_player3_brawler_Class,battle_player1_brawler_Role,battle_player2_brawler_Role,battle_player3_brawler_Role,battle_power,avg_brawler_trophies,brawler_trophies_min,brawler_trophies_max,avg_trophies,trophies_min,trophies_max,avg_highestTrophies,highestTrophies_min,highestTrophies_max,avg_brawler_Health,brawler_Health_min,brawler_Health_max,avg_brawler_winrate,brawler_winrate_min,brawler_winrate_max,avg_brawler_winrate_overall,brawler_winrate_overall_min,brawler_winrate_overall_max,avg_brawler_userate,brawler_userate_min,brawler_userate_max,avg_brawler_userate_overall,brawler_userate_overall_min,brawler_userate_overall_max,avg_team_victories,team_victories_min,team_victories_max,avg_expPoints,expPoints_min,expPoints_max
0,True,60,18,82,29,0.0,gemGrab,Deep Diner,MR. P,TARA,BIBI,Damage Dealer,Damage Dealer,Damage Dealer,area control,area control,area control,33.0,6.918036,6.715383,7.084226,10.888228,10.505451,11.263553,10.888352,10.505999,11.263553,8.594339,8.412055,8.839422,0.490730,0.452415,0.547120,0.502476,0.477947,0.535703,0.740007,0.576903,1.063193,1.031029,0.263347,1.698714,10.177616,8.684570,11.010002,12.858840,12.023405,13.526012
1,False,60,18,82,29,0.0,gemGrab,Deep Diner,SANDY,GRAY,FRANK,Support,Support,Tank,area control,sniper,tank,33.0,6.996376,6.976348,7.035269,10.808427,10.642874,11.074343,10.845102,10.643280,11.156265,8.860925,8.507345,9.259226,0.538669,0.476378,0.597090,0.521462,0.473367,0.564274,0.666006,0.191798,1.193827,0.838300,0.352721,1.349106,9.846952,8.404472,10.560697,12.632252,11.993275,13.292730
2,True,60,18,82,29,0.0,gemGrab,Deep Diner,SANDY,GRAY,FRANK,Support,Support,Tank,area control,sniper,tank,33.0,6.987798,6.966967,7.028201,10.808427,10.642874,11.074343,10.845102,10.643280,11.156265,8.860925,8.507345,9.259226,0.538669,0.476378,0.597090,0.521462,0.473367,0.564274,0.666006,0.191798,1.193827,0.838300,0.352721,1.349106,9.846952,8.404472,10.560697,12.632252,11.993275,13.292730
3,False,60,18,82,29,0.0,gemGrab,Deep Diner,GALE,CHESTER,BONNIE,Hybrid,Damage Dealer,Hybrid,area control,assassin,sniper,32.0,6.841972,6.813445,6.857514,10.857575,10.588880,11.074343,10.892366,10.588880,11.156265,8.674368,8.507345,8.881975,0.481797,0.404844,0.524731,0.505076,0.461216,0.550531,0.527318,0.218226,1.053377,0.602900,0.344665,0.997353,9.952182,9.407058,10.560697,12.879975,12.533576,13.292730
4,False,60,18,82,29,0.0,gemGrab,Deep Diner,GALE,CHESTER,GRAY,Hybrid,Damage Dealer,Support,area control,assassin,sniper,32.0,6.784457,6.651572,6.864848,10.857575,10.588880,11.074343,10.892366,10.588880,11.156265,8.537192,8.507345,8.594339,0.508889,0.404844,0.597090,0.509657,0.461216,0.564274,0.821810,0.218226,1.193827,0.937714,0.466684,1.349106,9.952182,9.407058,10.560697,12.879975,12.533576,13.292730


In [ ]:
mode_map = battlelog[['event_mode','event_map']].drop_duplicates().reset_index(drop=True)
mode_map

In [ ]:
brawlers_stats = ks.read_parquet('datasets/brawlers/brawlers_stats.parquet')
brawlers_stats.head()

In [ ]:
avg_rank = battlelog[['avg_brawler_trophies','avg_trophies','avg_highestTrophies','avg_team_victories','avg_expPoints']].agg(np.mean)
avg_rank

In [ ]:
combinations = pd.DataFrame(itertools.combinations(brawlers_stats['name'], 3))
combinations.columns = ['battle_player1_brawler_name', 'battle_player2_brawler_name', 'battle_player3_brawler_name']
combinations

In [ ]:
predictor = pd.DataFrame()

for i, item in enumerate(mode_map['event_mode']):
	combinations['event_mode'] = mode_map.loc[i, 'event_mode']
	combinations['event_map'] = mode_map.loc[i, 'event_map']
	predictor = pd.concat([predictor, combinations], ignore_index=True)

predictor.reset_index(drop=True)
predictor

In [ ]:
predictor['battle.mode'] = predictor['event.mode']
predictor['battle.type'] = 'ranked'
predictor['battle.player1.trophies'] = avg_rank.trophies
predictor['battle.player1.brawler.trophies'] = avg_trophies
predictor['battle.player1.brawler.power'] = 11
predictor['battle.player2.trophies'] = avg_rank.trophies
predictor['battle.player2.brawler.trophies'] = avg_trophies
predictor['battle.player2.brawler.power'] = 11
predictor['battle.player3.trophies'] = avg_rank.trophies
predictor['battle.player3.brawler.trophies'] = avg_trophies
predictor['battle.player3.brawler.power'] = 11

In [ ]:
predictor = predictor.merge(brawlers_stats.add_prefix('battle.player1.brawler.'), on='battle.player1.brawler.name')
predictor = predictor.merge(brawlers_stats.add_prefix('battle.player2.brawler.'), on='battle.player2.brawler.name')
predictor = predictor.merge(brawlers_stats.add_prefix('battle.player3.brawler.'), on='battle.player3.brawler.name')

In [ ]:
predictor

In [ ]:
predictions = predict_model(model, data = predictor, raw_score=True)

In [ ]:
predictions['Label'].value_counts()

In [ ]:
predictions.sort_values(by='Score_y', ascending=False)